In [1]:
import os
import gzip

import numpy as np
# import pandas as pd
# from PIL import Image
from IPython import display

import time
# from tqdm import tqdm_notebook
# from tqdm import trange

# import keras
from keras import backend as K
from keras.callbacks import Callback
from keras.models import Model, load_model
from keras.optimizers import SGD, Adam
from keras.layers import Dense, Dropout, GlobalAveragePooling2D # Conv2D, Input, Flatten, MaxPooling2D, UpSampling2D, concatenate, Cropping2D, Reshape, BatchNormalization
from keras.utils import HDF5Matrix
from keras.applications.vgg16 import VGG16
# from keras.metrics import categorical_crossentropy
# from keras.utils.data_utils import get_file
# from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array
# from keras.utils import to_categorical

# from keras.applications.mobilenet import preprocess_input

from keras.models import load_model
import matplotlib.pyplot as plt
%matplotlib inline



Using TensorFlow backend.


In [2]:
from preprocess import DataGenerator

data_dir = './data/'

In [3]:
pc_test_x_h5 = gzip.open(data_dir + 'camelyonpatch_level_2_split_test_x.h5.gz', 'rb')
pc_test_y_h5 = gzip.open(data_dir + 'camelyonpatch_level_2_split_test_y.h5.gz', 'rb')

In [4]:
x_test = HDF5Matrix(pc_test_x_h5, 'x')
y_test = HDF5Matrix(pc_test_y_h5, 'y')

In [16]:
len(x_test)

32768

In [6]:
# directory to save the best model
file_dir = './Model'
if not os.path.exists(file_dir):
    os.mkdir(file_dir)
    
model_name = 'vgg16_model_10k'
    
network_filepath = os.path.join(file_dir, model_name + '.h5') #_limitless

In [7]:
model = load_model(network_filepath)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [13]:
# indexes
test_id = np.arange(len(x_test))

# create a useful dictionary structures
partition = {}
partition['test'] = test_id
    
test_labels = {str(i) : y_test[i].flatten()[0] for i in test_id}

In [17]:
# Define batch size.
batch_size = 128

# Parameters for generators
params = {
    'dim': (224, 224),
    'batch_size': batch_size,
    'n_classes': 2,
    'shuffle': False
}

# Generators
test_generator = DataGenerator.DataGenerator(partition['test'], x_test, test_labels, **params)

In [18]:
preds = model.predict_generator(test_generator)

In [ ]:
preds

In [26]:
type(list(preds))

list

In [36]:
with open(os.path.join(file_dir, 'preds_vgg16_10k.csv'), 'w') as f:
    f.write("case, prediction1, prediction2")
    for i, p in enumerate(preds):
        f.write(str(i) + ',' + str(p[0]) + ',' + str(p[1]) + '\n')

In [38]:
with open(os.path.join(file_dir, 'preds_vgg16_10k_sub.csv'), 'w') as f:
    f.write("case, prediction\n")
    for i, p in enumerate(preds):
        f.write(str(i) + ',' + str(p[1]) + '\n')